In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import networkx as nx
import random
from ortools.graph.python import max_flow
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Génération de réseaux et test de l'implémentation de l'algorithme Push-Relabel de Goldberg Tarjan**:
### Variante de l'étiquette la plus élevée

Afin de vérifier l'exactitude de l'implémentation de l'algorithme Push-Relabel de Goldberg Tarjan (1988), nous la comparerons avec l'implémentation de l'outil Google OR-Tools (https://developers.google.com/optimization?hl=fr) du même algorithme.

In [2]:
random.seed(42)

#### Génération de réseaux

In [3]:
def add_capacity(Network, minimum_capacity, maximum_capacity):
    """
    Network: Networkx object of class Digraph
    minimum_capacity: the min of the capacity of an edge
    maximum_capacity: the max of the capacity of an edge
    """
    network = Network.copy()
    for u,v,capacity in network.edges(data="capacity"):
        if capacity is None:
            network.edges[u,v]["capacity"] = random.randint(minimum_capacity,maximum_capacity)
    return network

In [4]:
def generate_networks(number_of_networks, number_of_nodes, minimum_capacity, maximum_capacity, probability_of_link):
    """
    returns a list of size number_of_networks of adjacency matrices of size number_of_nodes x number_of_nodes
    minimum_capacity: the min of the capacity of an edge
    maximum_capacity: the max of the capacity of an edge
    probability_of_link: the probability of a link being present in the network
    """
    #list_of_seed = list(np.random.randint(30,high=100,size=number_of_networks))
    list_of_adj = []
    
    for _ in range(number_of_networks):
        #print(s)
        random_network = nx.gnp_random_graph(number_of_nodes, probability_of_link, seed = None, directed = True)
        #print(random_network.edges())
        random_network = add_capacity(random_network, minimum_capacity, maximum_capacity)
        #nx.draw(random_network)
        #plt.show()
        list_of_adj.append(nx.adjacency_matrix(random_network, weight = 'capacity').toarray())
    return list_of_adj

In [5]:
list_of_adj = generate_networks(10000, 16, 1, 10, 0.2)
print(len(list_of_adj))

10000


Conformément à la documentation du CLRS: https://github.com/deepmind/clrs, pour l'entraînement, 1000 réseaux composés de 16 noeuds seront utilisés. En guise de test, 10000 seront utilisés.

In [6]:
print(type(list_of_adj[0]))
print(list_of_adj[0])

<class 'numpy.ndarray'>
[[ 0  0  7  0  0  0  0  0 10  0 10  0  0  9  6  0]
 [ 0  0  0  0  0  5  0  0  0  4  0  0  6  4  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  5  0  0  7]
 [ 0  3  0  0  0  0  0  0  0  0  0  0  5  0  8  0]
 [ 0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  1]
 [ 8  0  0  0 10  0  0  0  0  0  0 10  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  2  0  0  0  0  0  9]
 [ 0  0  0  0  4  0  9  0  0  0  0  0  5  0  0  3]
 [ 0  6  0  0  2  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  4  0  0  0  0  0  0  6  0  0  0  0]
 [ 5  0  0  0  3  0  0  0  0  0  8  0  9  0  0  0]
 [ 0  0  0  5  0  0  0  0 10  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0  1  0  0  0  9  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  2]
 [ 0  0  0  3  0  5  2  0  0  0  2  0  0  0  0  0]]


Chaque réseau sera représenté par une matrice d'adjacence et une liste d'incidence pour chaque sommet.

In [7]:
def adjacencyToIncidenceList(A):
    """
    takes an adjacency matrix
    returns a dict representing the incidence list of each vertex
    """
    Incidence_list = dict()
    tp_list = []
    for v in range(A.shape[0]):
        Incidence_list[v] = []
    
    for v in range(A.shape[0]):
        w_s = np.argwhere(A[v,:]>0)
        for w in w_s:
            if (v,w[0]) not in Incidence_list[v]:
                Incidence_list[v].append((v,w[0]))
            if (w[0],v) not in Incidence_list[w[0]]:
                Incidence_list[w[0]].append((w[0],v))
        #Incidence_list[v] = [tp_list,-1]# -1 for current edge data structure
        #tp_list = []
    return Incidence_list

Google OR-Tools a besoin en entrée de la liste des sommets émetteurs, de la liste des sommets récepteurs, et de la liste des capacités pour chaque arc ainsi formé.

In [8]:
def adjacencyToListORTools(A):
    """
    takes an adjacency matrix
    returns three list of start node, end node and capacities
    """
    start_node = []
    end_node = []
    capacities = []
    tp_list = []
    #for v in range(A.shape[0]):
        #Incidence_list[v] = []
    
    for v in range(A.shape[0]):
        w_s = np.argwhere(A[v,:]>0)
        for w in w_s:
            start_node.append(v)
            end_node.append(w[0])
            capacities.append(A[v,w[0]])
        #Incidence_list[v] = [tp_list,-1]# -1 for current edge data structure
        #tp_list = []
    return start_node,end_node,capacities

In [9]:
def googleORToolsMaxflow(A, s, t):
    """
    A capacity matrix
    returns maxflow and s-t cut
    """
    smf = max_flow.SimpleMaxFlow()
    start_nodes,end_nodes,capacities = adjacencyToListORTools(A)
    all_arcs = smf.add_arcs_with_capacity(start_nodes, end_nodes, capacities)
    # Find the maximum flow between node 0 and node 7.
    status = smf.solve(s, t)
    if status != smf.OPTIMAL:
        print('There was an issue with the max flow input.')
        print(f'Status: {status}')
        exit(1)
    maxflow = smf.optimal_flow()
    #print('Max flow:', maxflow)
    #print('')
    #print(' Arc    Flow / Capacity')
    #solution_flows = smf.flows(all_arcs)
    #for arc, flow, capacity in zip(all_arcs, solution_flows, capacities):
    #print(f'{smf.tail(arc)} / {smf.head(arc)}   {flow:3}  / {capacity:3}')
    S = smf.get_source_side_min_cut()
    T = smf.get_sink_side_min_cut()
    S.sort()
    T.sort()
    #print('Source side min-cut:', S)
    #print('Sink side min-cut:', T)
    return maxflow,S,T
    

#### Implémentation personnelle de l'algorithme Push-Relabel (variante de l'étiquette la plus élevée)

In [10]:
def push(v, w, f, r_f, e):
    """
    v,w: edge through which we will push some flow
    f: preflow (np 2D array)
    r_f: residual capacity (np 2D array)
    e: excess vector (np array)
    returns f,r_f,e
    """
    f = f.copy()
    r_f = r_f.copy()
    e = e.copy()
    delta = min(e[v], r_f[v,w])
    f[v,w] += delta
    f[w,v] -= delta
    r_f[v,w] -= delta
    r_f[w,v] += delta
    e[v] -= delta
    e[w] += delta
    return f,r_f,e
    

In [11]:
def relabel(v, r_f, d):
    """
    relabelling operation
    v: vertex
    r_f: residual capacity (np 2D array)
    d: label
    returns d
    """
    w_s = np.argwhere(r_f[v,:]> 1e-6)
    d = d#.copy()
    if len(w_s) > 0:
        mini = np.min(d[w_s])
        #print(mini)
        d[v] = mini + 1
    return d

In [12]:
def procedure(A, C, Q, v, s, t, f, r_f, e, d, current_edge, rel):
    """
    A: dict of the incidence lists of each vertex, with G being a symmetric digraph
    C: capacity matrix
    Q: priority queue
    v: removed vertex with highest label priority in Q
    s: source node
    t: sink node
    f: preflow
    r_f: residual capacity
    e: excess vector
    d: label vector
    rel: a boolean variable associated with the highest label push algorithm
    
    returns f, r_f, e, d, rel
    """
    A = A.copy()
    Q = Q.copy()
    f = f.copy()
    r_f = r_f.copy()
    e = e.copy()
    d = d.copy()
    current_edge = current_edge.copy()
    rel = rel
    # Current edge in A_v
    v,w = A[v][current_edge[v]]
    #print(v,w)

    inside_queue = False
    # Adding for underflow 
    if(C[v,w] - f[v,w] > 1e-6 and (d[v]==d[w]+1)):

        #print("e before", e)
        f, r_f, e = push(v, w, f, r_f, e)
        #print("e after", e)

        for priority,vertex in Q:
            if(w==vertex):
                inside_queue = True
        if((not inside_queue) and w != s and w != t):

            Q.append((-d[w],w))
            Q.sort(key=lambda vertex: vertex[0])#sort by priority

    elif e[v] > 1e-6:

        if current_edge[v] < len(A[v])-1:

            current_edge[v] = current_edge[v] + 1 

        else:

            d = relabel(v, r_f, d)
            #print(d)
            rel = True
            # first edge as current edge
            current_edge[v] = 0

    return A,Q,f,r_f,e,d,current_edge,rel

In [13]:
def HLFlow(A, C, s, t):
    """
    Highest Label flow push algorithm, Goldberg and Tarjan 1988
    A: Incidence lists
    C: capacity matrix
    s: source node
    t: sink node
    return flow_value
    """
    # Initialization of all variables
    #preflow
    n,m = C.shape
    f = np.full((n,m), 0.)
    for v in range(n):
        f[s,v] = C[s,v]
        f[v,s] = -C[s,v]
    # labels and excess
    d = np.array([0 for i in range(n)])
    e = np.array([0.0 for i in range(n)])
    d[s] = n
    for v in range(n):
        if v!= s:
            e[v] = f[s,v]
            
    # residual capacity
    r_f = C - f
    for v in range(n):
        if v != s:
            r_f[s,v] = 0.
            r_f[v,s] = C[v,s] + C[s,v]
    # priority queue and current edge for each A_v
    Q = []
    current_edge = np.array([0 for i in range(n)])
    for v in range(n):
        if v!= s:
            # make the first edge the current edge
            current_edge[v] = 0
            # pushing in Q
            if e[v] > 0 and v != t:
                Q.append((-d[v],v))
    Q.sort(key=lambda vertex: vertex[0])# sort by priority
    while len(Q) > 0:
        # remove vertex of highest priority
        priority,v = Q.pop(0)
        rel = False
        A,Q,f,r_f,e,d,current_edge,rel = procedure(A, C, Q, v, s, t, f, r_f, e, d, current_edge,rel)
        while(e[v] > 0. and rel==False):
            A,Q,f,r_f,e,d,current_edge,rel = procedure(A, C, Q, v, s, t, f, r_f, e, d, current_edge, rel)
        #if e[v] > 0 and d[v] < n and -d[v] < priority:
        if e[v] > 0 :
            Q.append((-d[v],v))
            Q.sort(key=lambda vertex: vertex[0])# sort by priority
            
    S = []
    T = []
    target_height = -1
    for height in range(1,n):
        if height not in d:
            target_height = height
            #break
    if target_height != -1:
        for vertex in np.argwhere(d > target_height):
            S.append(vertex[0])
        
        for vertex in np.argwhere(d < target_height):
            T.append(vertex[0])
    else:
        S = []
        T = []
    return e[t],S,T

#### Petits tests

In [14]:
C1 = np.array(
    [
    [0,13,15,0],
    [0,0,7,12],
    [0,8,0,20],
    [0,0,0,0]
    ]
    )

A2 = adjacencyToIncidenceList(C1)
print(A2)

{0: [(0, 1), (0, 2)], 1: [(1, 0), (1, 2), (1, 3)], 2: [(2, 0), (2, 1), (2, 3)], 3: [(3, 1), (3, 2)]}


In [15]:
C2 = np.array(
    [
    [0,38,1,2,0,0,0,0],
    [0,0,8,0,13,0,10,0],
    [0,0,0,0,0,0,26,0],
    [0,0,0,0,0,0,0,27],
    [0,0,2,0,0,1,0,7],
    [0,0,0,0,0,0,0,7],
    [0,0,0,24,0,8,0,1],
    [0,0,0,0,0,0,0,0]
    ]
    )

A3 = adjacencyToIncidenceList(C2)
print(A3)

{0: [(0, 1), (0, 2), (0, 3)], 1: [(1, 0), (1, 2), (1, 4), (1, 6)], 2: [(2, 0), (2, 1), (2, 6), (2, 4)], 3: [(3, 0), (3, 7), (3, 6)], 4: [(4, 1), (4, 2), (4, 5), (4, 7)], 5: [(5, 4), (5, 7), (5, 6)], 6: [(6, 1), (6, 2), (6, 3), (6, 5), (6, 7)], 7: [(7, 3), (7, 4), (7, 5), (7, 6)]}


In [16]:
flow_value4 = HLFlow(A3, C2, 0, 7)
print(flow_value4)
flow_value4 = googleORToolsMaxflow(C2, 0, 7)
print(flow_value4)

(31.0, [0, 1, 4], [2, 3, 5, 6, 7])
(31, [0, 1, 4], [2, 3, 5, 6, 7])


In [17]:
flow_value1 = HLFlow(A2, C1, 0, 3)
googleORToolsMaxflow(C1, 0, 3)

(28, [0], [1, 2, 3])

In [18]:
print(flow_value1)

(28.0, [0], [1, 2, 3])


In [19]:
A = adjacencyToIncidenceList(list_of_adj[1])
flow_value = HLFlow(A, list_of_adj[1], 0, list_of_adj[1].shape[0]-1)

In [20]:
print(flow_value)

(3.0, [0, 11, 14], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15])


In [21]:
googleORToolsMaxflow(list_of_adj[1], 0, list_of_adj[1].shape[0]-1)

(3, [0, 11, 14], [1, 2, 3, 4, 5, 7, 9, 10, 12, 13, 15])

In [22]:
G=nx.from_numpy_matrix(list_of_adj[4])
fvalue = nx.maximum_flow_value(G,0,list_of_adj[4].shape[0]-1,capacity='weight')

In [23]:
G1=nx.from_numpy_matrix(C2)
fvalue1 = nx.minimum_cut(G1,0,C2.shape[0]-1,capacity='weight',flow_func=nx.algorithms.flow.preflow_push)

In [24]:
print(fvalue1)

(31, ({0, 1, 4}, {2, 3, 5, 6, 7}))


In [25]:
flow_value4 = HLFlow(A3, C2, 0, 7)
print(flow_value4)

(31.0, [0, 1, 4], [2, 3, 5, 6, 7])


In [26]:
print(fvalue)

32


In [27]:
print(list_of_adj[4])

[[ 0  0  0  0  0  0  8  0  0  4  0  6  0  0  0  0]
 [10  0  0  0  0  0  0  0  0  0  0  0  0  8  0  0]
 [ 0  0  0  8  5  0  7  0  0  9  0  0  0  9  0  0]
 [ 0  0  0  0  7  0  3  4  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  3  0  0  0  0  0  5  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  8  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  9  0  0  2  0  0]
 [ 0  0  9  0  0  2  0  0  0  0  0  5  2  0  3  5]
 [ 0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  3  0  0  0  0  0  7  0  0]
 [ 0  0  0  2  0  0  4  0  0  0  0  0  0  0  0  8]
 [ 0  6  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  7  1  0  0  0  0  0  0  7  0]
 [ 9  0  0  0  6  0  0  0  0  4  0  0  7  0  0  0]
 [ 0  0  2  0  6  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  1  0  0  0  0  6  0  0  0  0  0  0  0  0  0]]


In [28]:
A3 = adjacencyToIncidenceList(list_of_adj[4])
flow_value2 = HLFlow(A3, list_of_adj[4], 0, list_of_adj[4].shape[0]-1)
print(flow_value2)

(18.0, [0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])


In [29]:
G=nx.from_numpy_matrix(list_of_adj[4])
fvalue = nx.minimum_cut(G,0,list_of_adj[4].shape[0]-1,capacity='weight',flow_func=nx.algorithms.flow.preflow_push)
print(fvalue)

(32, ({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14}, {10, 15}))


In [30]:
googleORToolsMaxflow(list_of_adj[4], 0, list_of_adj[4].shape[0]-1)

(18, [0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

### Tests face à Google OR-Tools sur les 1000 réseaux générés

In [31]:
def computeCut(C,S,T):
    cut = 0.
    for u in S:
        for v in T:
            cut += C[u,v]
    return cut

In [32]:
def test_HLFlowVsGoogleOR(list_of_adjacency):
    counter_diff_cut = 0
    counter_diff_flow_value = 0
    diff_cut = 0
    print("testing " + str(len(list_of_adjacency)) + " networks")
    for adjacency_matrix in list_of_adjacency:
        A = adjacencyToIncidenceList(adjacency_matrix)
        s = random.randrange(adjacency_matrix.shape[0])
        t = random.randrange(adjacency_matrix.shape[0])
        while s==t:
            t = random.randrange(adjacency_matrix.shape[0])
        flow_value = HLFlow(A, adjacency_matrix, s, t)
        flow_value1 = googleORToolsMaxflow(adjacency_matrix, s, t)
        
        if(flow_value[1] != flow_value1[1]):
            print(flow_value[1])
            print(flow_value1[1])
            print("cut HLFLOW",flow_value[2])
            print("cut OR Tools",flow_value1[2])
            print(flow_value1[0] == flow_value[0])
            print(flow_value1[0])
            counter_diff_cut += 1
            cut = computeCut(adjacency_matrix,flow_value[1],flow_value[2])
            cut1 = computeCut(adjacency_matrix,flow_value1[1],flow_value1[2])
            if cut != cut1:
                diff_cut += 1
                print(cut,cut1)
            if flow_value[0] != flow_value1[0]:
                counter_diff_flow_value += 1
    print("diff cut", counter_diff_cut)
    print("diff flow",counter_diff_flow_value)
    print("diff cut value",diff_cut)

In [33]:
test_HLFlowVsGoogleOR(list_of_adj)

testing 10000 networks
[15]
[]
cut HLFLOW [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
cut OR Tools []
True
0
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14]
[]
cut HLFLOW [6, 11, 15]
cut OR Tools []
True
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[]
cut HLFLOW [15]
cut OR Tools []
True
0
diff cut 3
diff flow 0
diff cut value 0


Nous voyons ici que sur les 10000 réseaux, il n'y a eu que 3 réseaux où la coupe donnée par l'implémentation personnelle de l'algorithme de Goldberg Tarjan et celle de Google OR-Tools a différé (indiqué par **diff cut**). Cependant, aucune différence en terme de valeur du flot maximum, ni de différence pour la capacité de la coupe minimum n'a été trouvé. 

### ***Étapes suivantes***
#### 1. Adaptation de l'implémentation aux exigences du CLRS-30:
Le CLRS ne prend en charge que 5 types de structures de données:
* scalaire: nombre réels
* catégorielle: indiquant l'appartenance ou non à K différentes catégories
* mask: indiquant l'appartenance ou non à deux catégories (1 ou 0)
* mask_one: indiquant l'appartenance ou non à deux catégories (1 ou 0) où un seul sommet est actif
* pointeur

Chaque variable de l'algorithme devra être de l'un de ces types, il faudra également spécifié si la variable est une variable d'entrée, de sortie ou une variable indiquant l'état interne de l'algorithme.

En guise d'exemple, soient C, la matrice d'adjacence et de capacité d'un réseau, s, le sommet source, et t, le sommet puit.
C sera une variable d'entrée dans la catégorie scalaire car les capacités sont des nombres réels, s sera représenté comme suit:
Représentons les sommets en tant que vecteur, les sommets seront numérotés de 0 à n-1, pour n sommets.
la représentation de s sera:

sommets: 0 1 2 3 4 5 6 7 ... s ... n-1      
état: &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;0 0 0 0 0 0 0 0 ... 1 ... 0

t sera représenté de même. Ainsi, s et t sont dans la catégorie mask_one.

#### 2. Collecte des données d'entraînements
Une fois les variables dans le format correct, il faudra enregistrer l'évolution de chaque variable au cours de l'execution de l'algorithme, formant ainsi une trajectoire pour chaque variable.

#### 3. Entraînement des réseaux de neurones
Le CLRS fournit six architectures de réseaux de neurones, une fois les données acquises, elle seront fournis au module de génération de données d'entraînement du CLRS puis seront passées aux modèles pour la phase d'entraînement.